In [1]:
import asyncio # Asenkron programlama için kütüphane 
import requests # HTTP istekleri yapmak için kütüphane
import pandas as pd # Veri analizi ve tablo işlemleri için kütüphane
import gradio as gr #kullanıcı arayüzü oluşturan kütüphane
import cv2 # Görüntü işleme işlemleri için kütüphane
import numpy as np  # Matematiksel işlemler için kütüphane
from tensorflow.keras.models import load_model # Önceden eğitilmiş modelleri yüklemek için kütüphane
from PIL import Image # Görüntülerle çalışmak için kütüphane
from io import BytesIO # Bellekte ikili veri yönetimi için kütüphane 
import json # JSON veri işlemleri için json kütüphanesi



In [2]:


# Önceden eğitilmiş Continuum modeli yüklenir
continuum_cnn1 = load_model("continuum_cnn1.h5")
continuum_class_names = ['continuum-alpha', 'continuum-beta']

def predict_continuum_image(img):
    """Continuum görseli için tahmin yapar."""
    img_resized = cv2.resize(img, (256, 256))  # Görseli modele uygun olacak şekilde yeniden boyutlandırır
    img_resized = img_resized / 255.0  # Normalizasyon
    prediction = continuum_cnn1.predict(np.array([img_resized]))[0]
    return {continuum_class_names[i]: float(prediction[i]) for i in range(len(continuum_class_names))}

# Önceden eğitilmiş Magnetogram modeli yüklenir
magnetogram_cnn1 = load_model("magnetogram_cnn1.h5")
magnetogram_class_names = ['magnetogram-alpha', 'magnetogram-beta']

def predict_magnetogram_image(img):
    """Magnetogram görseli için tahmin yapar."""
    img_resized = cv2.resize(img, (256, 256))  # Görseli modele uygun olacak şekilde yeniden boyutlandırır
    img_resized = img_resized / 255.0  # Normalizasyon
    prediction = magnetogram_cnn1.predict(np.array([img_resized]))[0]
    return {magnetogram_class_names[i]: float(prediction[i]) for i in range(len(magnetogram_class_names))}



In [3]:
# API'den KP indeksi verilerini almak için URL
KP_URL = "https://services.swpc.noaa.gov/json/planetary_k_index_1m.json"

# KP değerini sınıflandırma fonksiyonu
def classify_kp(kp_value):
    """KP değerini sınıflandırır ve bir açıklama döndürür."""
    if kp_value == 5:
        return "G1 (Minör): Güç sistemlerinde küçük dalgalanmalar olabilir."
    elif kp_value == 6:
        return "G2 (Orta): Gerilim alarmları olabilir."
    elif kp_value == 7:
        return "G3 (Şiddetli): Bazı koruma cihazları yanlış alarm verebilir."
    elif kp_value == 8:
        return "G4 (Şiddetli): Yaygın gerilim kontrol sorunları yaşanabilir."
    elif kp_value == 9:
        return "G5 (Aşırı): Elektrik kesintileri ve ağ arızaları yaşanabilir."
    else:
        return "Fırtına olasılığı düşük."

async def get_kp_value():
    try:
        resp = requests.get(KP_URL, timeout=10)  # URL'ye bağlanır, 10 saniye içinde veri alınmazsa hata fırlatır
        resp.raise_for_status()  # Veri geçerli değilse hata fırlatır
        data = resp.json()  # Yanıtı JSON formatına dönüştürür
        df = pd.DataFrame(data)  # JSON verisini DataFrame'e dönüştürür
        kp_val = int(df['kp_index'].iloc[-1])  # DataFrame'den en son KP indeks değerini alır
        return kp_val
    except Exception as e:
        return f"Hata: {e}"  # Bir şeyler ters giderse hata mesajı döndürür

def fetch_kp_sync():
    # Asenkron KP alma fonksiyonunu senkron olarak çalıştırır
    return asyncio.run(get_kp_value())



In [4]:
# Continuum ve Magnetogram verileri için URl'ler
CONTINUUM_URL = "https://sdo.gsfc.nasa.gov/assets/img/latest/latest_4096_HMIIC.jpg"
MAGNETOGRAM_URL = "https://sdo.gsfc.nasa.gov/assets/img/latest/latest_4096_HMIB.jpg"

# URL'den görseller alınabilirse görsel kırpılır, alınamazsa hata alır
def fetch_and_crop_image(url):
    """Verilen URL'den görseli alır ve kırpar."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # İstek hatası varsa kontrol et
        img = Image.open(BytesIO(response.content))
        img_np = np.array(img)

        # Görseli kırparak güneşin sadece ekvator bölgesi kullanılır
        if len(img_np.shape) >= 2:  # Grayscale veya RGB formatı
            height, width = img_np.shape[:2]
            start_row, start_col = int(height * 0.25), int(width * 0.1)
            end_row, end_col = int(height * 0.75), int(width * 0.9)
            cropped_image = img_np[start_row:end_row, start_col:end_col]
            return cropped_image
        else:
            raise ValueError("Geçersiz görsel formatı")
    except Exception as e:
        print(f"URL'den görsel alınırken hata oluştu: {e}")
        return None



In [5]:
# Continuum modelinin sonucunu uydu entegrasyonu için bir dosyaya kaydetme fonksiyonu
def save_continuum_result_to_file(img, file_path):
    """Continuum modelinin sonucunu verilen dosyaya kaydeder."""
    # Model tahminini al
    continuum_result = predict_continuum_image(img)
    
    # Sonucu dosyaya yaz
    try:
        with open(file_path, 'w') as file: #bu adda bir dosya varsa içeriğini silip yeni sonuçalrı yazıyor. yoksa oluşturup içine yazıyor.
            json.dump(continuum_result, file)
        print(f"Sonuç başarıyla {file_path} dosyasına kaydedildi.")
    except Exception as e:
        print(f"Dosyaya yazarken bir hata oluştu: {e}")

# Magnetogram modelinin sonucunu bir dosyaya kaydetme fonksiyonu
def save_magnetogram_result_to_file(img, file_path):
    """Magnetogram modelinin sonucunu verilen dosyaya kaydeder."""
    # Model tahminini al
    magnetogram_result = predict_magnetogram_image(img)
    
    # Sonucu dosyaya yaz
    try:
        with open(file_path, 'w') as file: #bu adda bir dosya varsa içeriğini silip yeni sonuçalrı yazıyor. yoksa oluşturup içine yazıyor.
            json.dump(magnetogram_result, file)
        print(f"Sonuç başarıyla {file_path} dosyasına kaydedildi.")
    except Exception as e:
        print(f"Dosyaya yazarken bir hata oluştu: {e}")



In [6]:
# Interface fonksiyonu, Gradio arayüzünü tanımlar.
def interface():
    # Continuum ve Magnetogram görüntülerini URL'den alıp kırpar.
    continuum_image = fetch_and_crop_image(CONTINUUM_URL)
    magnetogram_image = fetch_and_crop_image(MAGNETOGRAM_URL)

    # Gradio Blocks kullanarak bir arayüz oluşturuluyor.
    with gr.Blocks() as demo:
        # İlk satırda görseller ve mevcut KP indeks alanı yer alıyor.
        with gr.Row():
            # Continuum görseli yükleme alanı.
            continuum_input = gr.Image(type="numpy", label="Continuum Görseli", value=continuum_image)
            # Magnetogram görseli yükleme alanı.
            magnetogram_input = gr.Image(type="numpy", label="Magnetogram Görseli", value=magnetogram_image)
            # Mevcut KP indeksini göstermek için metin kutusu.
            kp_output = gr.Textbox(label="Mevcut KP İndeksi")

        # İkinci satırda model tahminleri için alanlar var.
        with gr.Row():
            # Continuum modeli tahmini için etiket alanı.
            continuum_output = gr.Label(label="Continuum Modeli Tahmini")
            # Magnetogram modeli tahmini için etiket alanı.
            magnetogram_output = gr.Label(label="Magnetogram Modeli Tahmini")
            # KP sınıflandırması için metin kutusu.
            kp_classification_output = gr.Textbox(label="KP Sınıflandırması")

        # Tahmin yapmak için bir düğme tanımlanıyor.
        predict_button = gr.Button("Tahmin Yap")

        # Tahmin fonksiyonu, girişleri alır ve sonuçları döner.
        def predict_all(cont_img, mag_img):
            # KP değerini senkron olarak al.
            kp_value = fetch_kp_sync()
            # KP sınıflandırmasını yap.
            kp_classification = classify_kp(kp_value)
            # Continuum görselini modelle tahmin et.
            continuum_result = predict_continuum_image(cont_img)
            # Magnetogram görselini modelle tahmin et.
            magnetogram_result = predict_magnetogram_image(mag_img)
            
            # Continuum sonucunu dosyaya kaydet.
            save_continuum_result_to_file(cont_img, "continuum_result.json")
            # Magnetogram sonucunu dosyaya kaydet.
            save_magnetogram_result_to_file(mag_img, "magnetogram_result.json")
            
            # Tüm sonuçları döndür.
            return continuum_result, magnetogram_result, kp_value, kp_classification

        # Düğmeye tıklanıldığında predict_all fonksiyonu çalışacak.
        predict_button.click(
            fn=predict_all,
            inputs=[continuum_input, magnetogram_input],
            outputs=[continuum_output, magnetogram_output, kp_output, kp_classification_output],
        )

    return demo  # Arayüzü döndürür.

# Arayüz başlatılır.
if __name__ == "__main__":
    interface().launch()  # Arayüzü başlatır.


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 18ms/step
Sonuç başarıyla continuum_result.json dosyasına kaydedildi.
1/1 [==============================] - 0s 17ms/step
Sonuç başarıyla magnetogram_result.json dosyasına kaydedildi.
